# 1.1 Reading a 2D scalar field from a simulation

In [ ]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

First we define the objects we will use - the mesh reader (to read the mesh from the FEM file), the pickle manager (to save the mesh data that we want to analyse), and the unfolder (to manage the mesh).

In [ ]:
from sim_reader import MeshReader, Unfolder
from object_reader import PickleManager
from regressors import LModel, CSModel
from fields import ScalarField
import numpy as np


# Define the necessary objects
reader = MeshReader('monoblock_out.e')
pickle_manager = PickleManager()
unfolder = Unfolder()

Next we read the simulation data. Change the sensor_region from `right` to whatever you called the region where sensors can be placed in your FEM file.

We get an array of 3D positions of all the nodes in that region in the mesh. In the monoblock file they all have x=0, but in your file they could all have z=5 or y=3 or whatever. The `compress_2D` function removes the x components so that the array becomes an array of 2D positions. You will have to redefine this function if you are analysing a different face or surface.

We read all of the `temperature` values from the points in the region, and use this to define a `ScalarField`. We can then save this field, and the grid of comparison positions.

In [ ]:
# Read the simulation data
sensor_region = 'right'
pos_3D = reader.read_pos(sensor_region)
pos_2D = unfolder.compress_2D(pos_3D)
bounds = unfolder.find_bounds(pos_2D)
grid = unfolder.generate_grid(bounds, 30, 30)

temps = reader.read_scalar(sensor_region, 'temperature').reshape(-1, 1)
temp_field = ScalarField(LModel, bounds)
temp_field.fit_model(pos_2D, temps)

# Save the simulation data
pickle_manager.save_file('simulation', 'temp_plane_field.obj', temp_field)
pickle_manager.save_file('simulation', 'temp_plane_points.obj', grid)

We know that the field is uniform in the horizontal direction so there is no point in analysing a 2D scalar field when we could be using a 1D scalar field, so instead we compress it further into a line.

We create a new `ScalarField` only this time it is 1D.

In [ ]:
# Now compress our nice 2D field even further into a 1D field
pos1 = (bounds[0][0], bounds[0][1])
pos2 = (bounds[0][0], bounds[1][1])
line_2D = unfolder.generate_line(pos1, pos2, 50)
line_temps = temp_field.predict_values(line_2D)
line_1D = unfolder.compress_1D(line_2D)

bounds_1D = np.array([line_1D[0], line_1D[-1]])
new_line_field = ScalarField(CSModel, bounds_1D)
new_line_field.fit_model(line_1D, line_temps)

# Save the new 1D line field
pickle_manager.save_file('simulation', 'temp_line_field.obj', new_line_field)
pickle_manager.save_file('simulation', 'temp_line_points.obj', line_1D)